In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Dataset_hyperEarlyStop/'

/content/drive/MyDrive/Dataset_hyperEarlyStop


In [4]:
eval_acc = pd.read_csv('dataset/eval_acc.csv')
eval_loss = pd.read_csv('dataset/eval_loss.csv')
train_loss = pd.read_csv('dataset/train_loss.csv', header = 1)
HP_space = pd.read_csv('dataset/HP_space.csv')

In [5]:
eval_acc

,L1,H1,L2,H2,epoch_1,epoch_2,epoch_3,epoch_4,epoch_5,epoch_6,...,epoch_141,epoch_142,epoch_143,epoch_144,epoch_145,epoch_146,epoch_147,epoch_148,epoch_149,epoch_150
0,3,4,0,0,0.4406,0.5187,0.5408,0.5667,0.5715,0.5954,...,0.6747,0.6711,0.6728,0.6693,0.6741,0.6778,0.6755,0.6765,0.6723,0.6712
1,3,5,0,0,0.4840,0.5188,0.5925,0.5950,0.6226,0.6209,...,0.7244,0.7220,0.7242,0.7213,0.7223,0.7243,0.7227,0.7230,0.7225,0.7228
2,3,6,0,0,0.4502,0.5097,0.5534,0.5717,0.5885,0.6129,...,0.6907,0.6872,0.6890,0.6874,0.6894,0.6884,0.6908,0.6875,0.6884,0.6903
3,3,7,0,0,0.4642,0.5356,0.5693,0.5785,0.6153,0.6260,...,0.7106,0.7103,0.7139,0.7145,0.7118,0.7117,0.7124,0.7111,0.7121,0.7133
4,3,8,0,0,0.4891,0.5453,0.5748,0.6139,0.6328,0.6514,...,0.7258,0.7287,0.7271,0.7307,0.7285,0.7293,0.7269,0.7275,0.7269,0.7289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,3,4,8,6,0.3824,0.4322,0.4656,0.4692,0.4921,0.5248,...,0.5811,0.5490,0.5802,0.5846,0.5748,0.5767,0.5671,0.5741,0.5644,0.5731
200,3,4,8,8,0.3723,0.4261,0.4466,0.4691,0.4793,0.5119,...,0.5728,0.5782,0.5674,0.5821,0.5930,0.5742,0.5602,0.5917,0.5815,0.5809
201,3,4,8,10,0.3998,0.4579,0.5098,0.5353,0.5305,0.5560,...,0.6320,0.6391,0.6345,0.6382,0.6403,0.6283,0.6342,0.6381,0.6399,0.6359
202,3,4,8,12,0.4094,0.4536,0.4823,0.4930,0.5193,0.5381,...,0.6413,0.6455,0.6371,0.6467,0.6488,0.6422,0.6432,0.6397,0.6399,0.6375


In [6]:
eval_loss

,L1,H1,L2,H2,epoch_1,epoch_2,epoch_3,epoch_4,epoch_5,epoch_6,...,epoch_141,epoch_142,epoch_143,epoch_144,epoch_145,epoch_146,epoch_147,epoch_148,epoch_149,epoch_150
0,3,4,0,0,1.52332,1.33248,1.27089,1.21834,1.20135,1.14159,...,1.02031,1.02160,1.02284,1.02495,1.04469,1.03849,1.02693,1.02176,1.02810,1.00954
1,3,5,0,0,1.42391,1.31039,1.14298,1.15141,1.05960,1.10058,...,0.90324,0.89469,0.90203,0.90636,0.89365,0.89999,0.91248,0.89115,0.89431,0.91435
2,3,6,0,0,1.51030,1.35697,1.24436,1.19246,1.16901,1.09636,...,0.98321,0.98734,0.99506,0.99040,0.98634,0.98861,0.98281,0.96875,0.98039,0.97649
3,3,7,0,0,1.49031,1.28391,1.21113,1.19441,1.09927,1.07175,...,0.92473,0.91630,0.90956,0.90825,0.92119,0.91781,0.91272,0.91210,0.91290,0.91833
4,3,8,0,0,1.42823,1.25468,1.18538,1.09234,1.04129,1.00743,...,0.88154,0.87421,0.87666,0.86612,0.86576,0.86915,0.87428,0.87241,0.88057,0.87955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,3,4,8,6,1.71504,1.58522,1.58067,1.51313,1.50656,1.43416,...,1.42088,1.40040,1.44347,1.51914,1.48245,1.38967,1.41427,1.38796,1.42785,1.50478
200,3,4,8,8,1.76575,1.59649,1.63956,1.52799,1.47745,1.41070,...,1.34234,1.32800,1.32745,1.31901,1.33631,1.31591,1.35848,1.33216,1.31453,1.33992
201,3,4,8,10,1.64943,1.51078,1.42041,1.33133,1.31672,1.32507,...,1.16164,1.17042,1.18172,1.19754,1.21205,1.15806,1.18568,1.13320,1.18360,1.19546
202,3,4,8,12,1.63926,1.55053,1.46117,1.41512,1.38881,1.31677,...,1.12982,1.14530,1.12501,1.13264,1.14269,1.13442,1.13576,1.15576,1.15447,1.13650


In [7]:
train_loss

,L1,H1,L2,H2,batch_1,batch_2,batch_3,batch_4,batch_5,batch_6,...,batch_41.149,batch_42.149,batch_43.149,batch_44.149,batch_45.149,batch_46.149,batch_47.149,batch_48.149,batch_49.149,batch_50.149
0,3,4,0,0,2.285,2.254,2.168,2.103,2.075,1.998,...,0.598,0.601,0.619,0.577,0.592,0.590,0.614,0.638,0.634,0.644
1,3,5,0,0,2.283,2.187,2.093,2.061,2.015,1.988,...,0.505,0.448,0.469,0.541,0.480,0.523,0.490,0.514,0.462,0.485
2,3,6,0,0,2.285,2.231,2.130,2.086,2.082,2.011,...,0.549,0.533,0.539,0.572,0.577,0.568,0.565,0.565,0.552,0.579
3,3,7,0,0,2.297,2.209,2.147,2.095,1.992,2.026,...,0.559,0.547,0.534,0.557,0.536,0.532,0.547,0.533,0.569,0.518
4,3,8,0,0,2.281,2.213,2.127,2.073,2.043,1.994,...,0.516,0.473,0.525,0.502,0.486,0.510,0.460,0.486,0.504,0.490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,3,4,8,6,2.295,2.202,2.143,2.100,2.083,2.042,...,0.805,0.849,0.793,0.743,0.816,0.752,0.788,0.810,0.797,0.829
198,3,4,8,8,2.295,2.226,2.160,2.100,2.096,2.059,...,0.825,0.774,0.815,0.787,0.841,0.857,0.862,0.836,0.803,0.800
199,3,4,8,10,2.300,2.230,2.180,2.145,2.111,2.027,...,0.690,0.639,0.687,0.674,0.705,0.727,0.704,0.658,0.697,0.692
200,3,4,8,12,2.307,2.231,2.175,2.132,2.065,2.046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
HP_space

,L1,H1,L2,H2
0,3,4,0,0
1,3,5,0,0
2,3,6,0,0
3,3,7,0,0
4,3,8,0,0
...,...,...,...,...
197,3,4,8,6
198,3,4,8,8
199,3,4,8,10
200,3,4,8,12


In [9]:
# Removing rows which has hyperparameters not belonging to the hyperparameter space
eval_acc_df = pd.merge(eval_acc, HP_space, on=['L1', 'H1', 'L2', 'H2'], how='inner')
eval_loss_df = pd.merge(eval_loss, HP_space, on=['L1', 'H1', 'L2', 'H2'], how='inner')
train_loss_df = pd.merge(train_loss, HP_space, on=['L1', 'H1', 'L2', 'H2'], how='inner')

In [10]:
# Interpolating missing values
eval_acc_df.interpolate(axis=1, inplace=True)
eval_loss_df.interpolate(axis=1, inplace=True)
train_loss_df.interpolate(axis=1, inplace=True)

In [11]:
train_loss_df.shape

(202, 7504)

In [12]:
# Aggregating the loss of the mini-batches for each epoch
train_loss_2d = np.array(train_loss_df.iloc[:, 4:]).reshape(-1,150,50).mean(axis=2)

In [13]:
train_loss_df = pd.DataFrame(train_loss_2d, columns = eval_acc_df.columns[4:])

In [14]:
train_loss_df

,epoch_1,epoch_2,epoch_3,epoch_4,epoch_5,epoch_6,epoch_7,epoch_8,epoch_9,epoch_10,...,epoch_141,epoch_142,epoch_143,epoch_144,epoch_145,epoch_146,epoch_147,epoch_148,epoch_149,epoch_150
0,1.80450,1.54360,1.42050,1.33836,1.26908,1.22096,1.16952,1.13088,1.08728,1.06124,...,0.61248,0.60748,0.61230,0.60868,0.61012,0.61070,0.61046,0.60910,0.60834,0.61230
1,1.75376,1.44356,1.31394,1.22598,1.15796,1.10530,1.04810,1.00616,0.96272,0.92490,...,0.49278,0.49394,0.49234,0.49246,0.49702,0.49482,0.50176,0.49344,0.49738,0.49230
2,1.79684,1.50790,1.37962,1.29922,1.23096,1.17964,1.12508,1.08654,1.04524,1.00820,...,0.57564,0.57206,0.57128,0.56894,0.57270,0.56658,0.56794,0.57460,0.57250,0.57316
3,1.77288,1.48226,1.34854,1.25930,1.19072,1.13940,1.08588,1.04724,1.00828,0.96774,...,0.54178,0.54388,0.54098,0.53854,0.54120,0.53982,0.54502,0.54274,0.53962,0.53982
4,1.73990,1.43634,1.29282,1.21294,1.14030,1.09298,1.04426,0.99806,0.95868,0.92482,...,0.50190,0.50562,0.50680,0.49996,0.50288,0.50872,0.50674,0.50060,0.50568,0.50094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,1.83082,1.57518,1.46098,1.38018,1.33344,1.28838,1.24838,1.21694,1.18978,1.15718,...,0.80160,0.79818,0.80018,0.79912,0.80508,0.80320,0.80080,0.79892,0.80616,0.79990
198,1.84684,1.60108,1.49028,1.42108,1.35814,1.31594,1.27206,1.23380,1.20670,1.18084,...,0.82264,0.82750,0.82486,0.82824,0.82528,0.82296,0.82638,0.82102,0.82062,0.82408
199,1.83730,1.56590,1.42932,1.34724,1.28120,1.22974,1.18460,1.14374,1.10424,1.06788,...,0.68256,0.67978,0.68174,0.68902,0.68134,0.68264,0.68878,0.68564,0.67574,0.68288
200,1.84690,1.60510,1.49576,1.42456,1.37116,1.32248,1.27750,1.24370,1.20298,1.18322,...,0.80900,0.80900,0.80900,0.80900,0.80900,0.80900,0.80900,0.80900,0.80900,0.80900


In [18]:
# Reading and parsing the train, validation, test split
with open('dataset/dataPartition.txt', 'r') as f:
    partition_lines = f.readlines()
  
train_lines = [(int(line.strip())-2) for line in partition_lines[0][8:-2].split(',')]
validation_lines = [(int(line.strip())-2) for line in partition_lines[1][13:-2].split(',')]
test_lines = [(int(line.strip())-2) for line in partition_lines[2][7:-2].split(',')]

In [19]:
# Saving the preprocessed dataset
train_loss_df.iloc[train_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/train_loss_training.csv")
eval_loss_df.iloc[train_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/eval_loss_training.csv")
eval_acc_df.iloc[train_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/eval_acc_training.csv")

In [20]:
train_loss_df.iloc[validation_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/train_loss_validation.csv")
eval_loss_df.iloc[validation_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/eval_loss_validation.csv")
eval_acc_df.iloc[validation_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/eval_acc_validation.csv")

In [21]:
train_loss_df.iloc[test_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/train_loss_testing.csv")
eval_loss_df.iloc[test_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/eval_loss_testing.csv")
eval_acc_df.iloc[test_lines].to_csv("/content/drive/MyDrive/Dataset_hyperEarlyStop/preprocessed_dataset/eval_acc_testing.csv")